In [37]:
import pandas as pd 
import numpy as np 
from sklearn.utils import shuffle

df = pd.DataFrame()
colnames=['Review','Sentiment'] 
df = pd.read_csv('Stemmedlines_reviews.csv',encoding='utf-8',names=colnames)
df.set_index(['Review', 'Sentiment'])
df = shuffle(df).reset_index(drop=True)

X_train=df.loc[:250,'Review'].values
X_test=df.loc[251:,'Review'].values
Y_train=df.loc[:250,'Sentiment'].values
Y_test=df.loc[251:,'Sentiment'].values
df.head(10)
# print(X_train)

,Review,Sentiment
0,గౌతమ్ ( రాజ్ తరుణ్ ) పుట్టుక అంధ అతను ఓ ఆశ్రమం...,0
1,హీరో కార్తీక్ రామ్ చరణ్ కుటుంబం కోసం ప్రాణాలైన...,1
2,సావిత్రి నందిత చిన్నప్పట్నుంచే పెళ్లి అంటే పిచ...,1
3,ఎర్ర శీను ( విజయ్ దేవరకొండ ) ఒక దొంగ జీవితం స్...,0
4,ఎస్కే శివం కార్తికేయన్కు నట కావాలన్నది ఆశయం తల...,1
5,ధర్మం నార రోహిత్ నిజాయితీపరుడు జైలర్ చెడు అస్స...,1
6,ఫ్రెండ్స్తో కలువు ఇంటలిజెంట్గా దొంగతనం చెయ్యి ...,1
7,స్నేహం కోసం ప్రాణమిచ్చు శోభన్ ( బెల్లంకొండ శ్ర...,0
8,ఈడో రకం ఆడ రకం సినిమా హిట్ అందుకో మంచు విష్ణు ...,0
9,రవి ( సుమంత్ అశ్విన్ ) పోలీస్ కావాలని ఆశపడు చి...,0


In [38]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

tokenizer_obj = Tokenizer()
Total_Reviews = df.loc[:,'Review'].values
tokenizer_obj.fit_on_texts(Total_Reviews)

#pad sequence
max_length= max([len(s.split()) for s in Total_Reviews])

#size of the vocabulary
vocabulary_size = len(tokenizer_obj.word_index) +1 
X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)

X_train_pad = pad_sequences (X_train_tokens,maxlen=max_length,padding="post")
X_test_pad = pad_sequences (X_test_tokens,maxlen=max_length,padding="post")

Total_Reviews_tokens = tokenizer_obj.texts_to_sequences(X_train)
Total_reviews_pad = pad_sequences(Total_Reviews_tokens,maxlen=max_length,padding="post")

print(X_train_pad)

[[ 302  158  479 ...    0    0    0]
 [  14  762  323 ...    0    0    0]
 [1243 1929 5492 ...    0    0    0]
 ...
 [2598   14 2429 ...    0    0    0]
 [ 611  992 1304 ...    0    0    0]
 [ 715  343   28 ...    0    0    0]]


In [39]:
#traiing the word2vec model
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.test.utils import get_tmpfile
from gensim import scripts
from smart_open import open
import re
f = open("Stemmedlines_reviews.txt","r")
data = f.read()

#sg defines if the parameter is skpgram or cbow
sentences_strings_ted = []
for line in data.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

sentences_ted = []
for sent_str in sentences_strings_ted:
	tokens = sent_str.split()
	sentences_ted.append(tokens)

Embedding_size =100 #embedding_size is the size of the embedding vector.

path = get_tmpfile("word2vec.model")
w2v_model = Word2Vec(sentences = sentences_ted,size=100, window=5, min_count=1, workers=4)


#building the vocabulary
vocabulary_words = w2v_model.wv.vocab.keys()
vocabulary = { x:i  for x,i in enumerate(vocabulary_words)}

print(vocabulary)
print(len(vocabulary))

embeddings = 1 * np.random.randn(len(vocabulary_words) + 1,Embedding_size)
embeddings[0] = 0 

for index,word in vocabulary.items():
#     embeddings[index] = np.random.uniform(-0.25,0.25,Embedding_size)
    embeddings[index] = w2v_model.wv[word]
        
    
print(embeddings.shape)





{0: 'చిన్నప్పుడు', 1: 'నుంచి', 2: 'గురుకులం', 3: 'చదువు', 4: 'అరవింద్', 5: 'నాను', 6: 'నిజాయితీ', 7: 'మారుపేరు', 8: 'పెరుగు', 9: 'అపరిచితుడులో', 10: 'రామ', 11: 'లాగా', 12: 'ఉండు', 13: 'అతను', 14: 'సమాజం', 15: 'ఎన్నో', 16: 'ఇబ్బంది', 17: 'పడు', 18: 'వచ్చు', 19: 'చివర', 20: 'వల్ల', 21: 'అతడు', 22: 'ప్రాణం', 23: 'పో', 24: 'పరిస్థితి', 25: 'ఇలాంటి', 26: 'స్థితి', 27: 'తాను', 28: 'మిత్ర', 29: 'కలువు', 30: 'అవినీతి', 31: 'వ్యతిరేకం', 32: 'పోరాటం', 33: 'మొదలుపెట్టు', 34: 'పరులు', 35: 'జాబితా', 36: 'తియ్యి', 37: 'వారు', 38: 'న్యాయం', 39: 'దిగు', 40: 'అరవింద్కు', 41: 'పోలిక', 42: 'మాయ', 43: 'కన్నన్', 44: 'పెద్ద', 45: 'సమస్య', 46: 'ఎదురవ్వు', 47: 'అతనేం', 48: 'చెయ్యి', 49: 'గెలిచాడా', 50: 'లేదా', 51: 'అన్న', 52: 'మిగతా', 53: 'కథ', 54: 'కథనం', 55: 'నేపథ్యం', 56: 'గత', 57: 'చాలా', 58: 'సినిమా', 59: 'జెండా', 60: 'కపిరాజుకు', 61: 'కొన్ని', 62: 'పోలికలున్నప్పటికీ', 63: 'తమిళ', 64: 'దర్శకుడు', 65: 'సముద్రఖని', 66: 'కొంచెం', 67: 'భిన్నం', 68: 'డీల్', 69: 'ప్రయత్నం', 70: 'ప్రధాన', 71: 'హైలైట్', 72: 'కాన

In [40]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,GRU
from keras.layers.embeddings import Embedding
from gensim.models import Word2Vec
from keras.regularizers import l2


model=Sequential()
model.add(Embedding(len(embeddings), Embedding_size ,weights=[embeddings] ,input_length=max_length, trainable=True))
# model.add(GRU(units=150,dropout = 0.2,recurrent_dropout=0.2))
model.add(LSTM(units=75, dropout=0.2, recurrent_dropout=0.2, return_sequences=False))
# model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='softmax'))
# model.compile(loss='squared_hinge',
#               optimizer='adadelta',
#               metrics=['accuracy'])

model.compile(loss = 'sparse_categorical_crossentropy' , optimizer = 'adadelta', metrics = ['accuracy'])

In [41]:
print("Training")

model.fit(X_train_pad,Y_train,epochs =50, batch_size =8, validation_data = (X_train_pad,Y_train),verbose=2)

Training
Train on 251 samples, validate on 251 samples
Epoch 1/50
 - 13s - loss: 1.6228 - accuracy: 0.4502 - val_loss: 0.7516 - val_accuracy: 0.4861
Epoch 2/50
 - 13s - loss: 0.7588 - accuracy: 0.4900 - val_loss: 0.7295 - val_accuracy: 0.4861
Epoch 3/50
 - 13s - loss: 0.7365 - accuracy: 0.4582 - val_loss: 0.7182 - val_accuracy: 0.4861
Epoch 4/50
 - 13s - loss: 0.7154 - accuracy: 0.5020 - val_loss: 0.7041 - val_accuracy: 0.4861
Epoch 5/50
 - 14s - loss: 0.7292 - accuracy: 0.5020 - val_loss: 0.8382 - val_accuracy: 0.4861
Epoch 6/50
 - 14s - loss: 0.7187 - accuracy: 0.5060 - val_loss: 0.6975 - val_accuracy: 0.4861
Epoch 7/50
 - 15s - loss: 0.7233 - accuracy: 0.5139 - val_loss: 0.6963 - val_accuracy: 0.5139
Epoch 8/50
 - 14s - loss: 0.7137 - accuracy: 0.5179 - val_loss: 0.6971 - val_accuracy: 0.5139
Epoch 9/50
 - 14s - loss: 0.7139 - accuracy: 0.4661 - val_loss: 0.7121 - val_accuracy: 0.5139
Epoch 10/50
 - 14s - loss: 0.7072 - accuracy: 0.4701 - val_loss: 0.7118 - val_accuracy: 0.5139
Epoc